# 인덱스와 슬라이스

### 음수 인덱스를 통해 뒤에서부터 접근하기

In [4]:
my_numbers = (4, 5, 3, 9)
print(my_numbers[-1])
print(my_numbers[-3])

9
5


### slice를 통한 특정 구간의 요소 구하기

In [11]:
my_numbers = (1, 1, 2, 3, 5, 8, 13, 21)
print(my_numbers[2:5])
print(my_numbers[:3])
print(my_numbers[3:])
print(my_numbers[::])
print(my_numbers[1:7:2])   # 1에서 7 사이의 위치에 있는 요소를 2칸씩 점프하여 가져옴

(2, 3, 5)
(1, 1, 2)
(3, 5, 8, 13, 21)
(1, 1, 2, 3, 5, 8, 13, 21)
(1, 3, 8)


In [13]:
# 시퀀스에 간격을 전달하는 것 = 슬라이스를 전달하는 것
# slice는 파이썬 내장 객체로 직접 빌드하여 전달도 가능
interval = slice(1, 7, 2)
print(my_numbers[interval])

(1, 3, 8)


In [14]:
interval = slice(None, 3)
print(my_numbers[interval] == my_numbers[:3])

True


## 자체 시퀀스 생성
- 슬라이스 객체를 만들어 시퀀스에 전달하는 방법은 __getitem__이라는 매직 메서드 덕분에 동작
- 시퀀스는 __getitem_과 __len__을 모두 구현하는 객체이므로 반복 가능
- 리스트, 튜플, 문자열은 표준 라이브러리에 있는 시퀀스 객체

In [33]:
class Items:
    def __init__(self, *values):
        self._values = list(values)
        
    def __len__(self):
        return len(self._values)
    
    def __getitem__(self, item):
        return self._values.__getitem__(item)

**위의 코드는 클래스가 표준 라이브러리 객체를 감싸는 래퍼인 경우**
- self._values = list(values)로 클래스가 리스트 래퍼
- getitem에서 동일한 메서드를 호출하여 호환성을 유지 (리스트의 getitem 함수를 그대로 호출)

In [34]:
item = Items(1,2,3)
print(item.__len__())
print(item.__getitem__(0))

3
1


### 래퍼나 내장 객체를 사용하지 않고 자체적인 시퀀스를 구현하는 경우
1. 범위로 인덱싱하는 결과는 해당 클래스와 같은 타입의 인스턴스일 것
1. slice에 의해 제공된 범위는 파이썬이 하는 것처럼 마지막 요소는 제외해야 함

In [35]:
#1번 사항
range(1, 100)[25:50] #결과도 range여야 함

range(26, 51)

# 컨텍스트 관리자 (Context manager)
##### 주요 동작의 전후에 작업을 실행하려고 할 때 유용
- 파일을 열면 파일 디스크립터 누수를 막기 위해 작업 종료 시 적절히 닫는 과정 필요
- 서비스나 소켓 연결에 대해서도 적절히 닫는 과정 필요  

<u>위와 같은 상황을 해결하는 일반적인 방법은 **finally** 블록에 정리 코드를 넣는 것</u>

```python
fd = open(filename)
try:
    process_file(fd)
finally:
    fd.close()
```

##### 위 코드에 대한 조금 더 파이썬스러운 방식은 아래와 같음
```python
with open(filename) as fd:
    process_file(fd)
```
###   
with문은 컨텍스트 관리자로 진입하게 해줌  
컨텍스트 관리자의 구성 메서드
1. &#95;&#95;enter&#95;&#95;
2. &#95;&#95;exit&#95;&#95;

with문은 &#95;&#95;enter&#95;&#95;메서드 호출 -> 메서드가 반환한 것을 as 이후에 지정된 변수에 할당  
해당 블록에 대한 마지막 문장이 끝나면 컨텍스트가 종료되며 컨텍스트 관리자 객체의 &#95;&#95;exit&#95;&#95; 메서드를 호출함  
_블록 내에 예외 or 오류가 있어도 &#95;&#95;exit&#95;&#95; 메서드는 호출됨_

<blockQuote>컨텍스트 관리자는 관심사를 분리하고 독립적으로 유지되어야하는 코드를 분리하는 좋은 방법</blockQuote>
<blockQuote>블록의 전후에 필요로하는 특정 논리를 제공하기 위해 자체 컨텍스트 관리자 구현 가능</blockQuote>

---

### 스크립트를 사용해 데이터베이스 백업을 하려는 예시
- 데이터베이스 실행 X 일 때에만 백업 가능
- 백업 위해서는 서비스 중지
- 백업 끝나면 프로세스 재시작

In [42]:
def stop_database():
    run("systemctl stop postgresql.service")
    
def start_database():
    run("systemctl start postgresql.service")
    
class DBHandler:
    def __enter__(self):
        stop_database()
        return self
    
    def __exit__(self, exec_type, ex_value, ex_traceback):
        start_database()
        
def db_backup():
    run("pg_dump database")

def main():
    with DBHandler():
        db_backup()

 **&#95;&#95;enter&#95;&#95; 메서드**
 - 반환 값이 쓸 데 없음
 - &#95;&#95;enter&#95;&#95; 함수에서 무언가를 반환 하는 것이 좋은 습관  
 
 
 **&#95;&#95;exit&#95;&#95; 메서드**
 - 예외를 파라미터로 받음
 - 특별한 작업을 할 필요가 없으면 아무것도 반환하지 않아도 됨
 
 > 실수로 __exit__에서 True를 반환하지 않도록 주의! 만일 True를 반환한다면 이것이 정말 원하는 결과인지, 그리고 그렇게 하는 충분한 이유가 있는 지 확인 필요
 

### 컨텍스트 관리자 구현
 **&#95;&#95;enter&#95;&#95; 메서드**와 **&#95;&#95;exit&#95;&#95; 메서드** 두가지 매직 메서드만 구현하면 해당 객체는 컨텍스트 관리자 프로토콜을 지원할 수 있음  
 위의 방법은 일반적이나 유일한 방법은 X

**&#60;todo&#62;**  
컨텍스트 관리자를 좀 더 간결하게 구현하는 방법   
표준 라이브러리 (contextlib 모듈 등)를 사용하여 보다 쉽게 구현하는 방법

<hr>

### Contextlib 모듈
컨텍스트 관리자를 구현하거나 더 간결한 코드를 작성하는 데 도움이 되는 많은 도우미 함수와 객체를 제공
### contextmanager 데코레이터
- 데코레이터를 적용한 함수의 코드를 컨텍스트 관리자로 변환
- 적용받는 함수는 **제너레이터** 라는 특수한 형태여야 함
- 제너레이터 함수는 코드의 문장을 **&#95;&#95;enter&#95;&#95;**와 **&#95;&#95;exit&#95;&#95;** 매직 메서드로 분리

In [50]:
import contextlib

@contextlib.contextmanager
def db_handler():
    stop_database()
    yield
    start_database()
    
def main():    
    with db_handler():
        db_backup()

**위의 코드는**  
- 제너레이터 함수 정의 후 @contextlib.contextmanager 데코레이터 적용
- db_handler는 yield문을 사용했으므로 제너레이터 함수가 됨
- 데코레이터를 적용하면 yield문 앞의 모든 것은 &#95;&#95;enter&#95;&#95; 메서드의 일부처럼 취급
    - &#95;&#95;enter&#95;&#95; 메서드의 반환값과 같은 역할을 하는 것으로 as x: 와 같은 형태로 변수에 할당 가능
- yield문 뒤의 모든 것은 &#95;&#95;exit&#95;&#95;의 로직
<hr>

### contextlib.ContextDecorator 클래스로 컨텍스트 관리자 만들기
컨텍스트 관리자 안에서 실행될 함수에 데코레이터를 적용하기 위한 로직을 제공하는 믹스인 클래스
> 믹스인 클래스 = 다른 클래스에서 필요한 기능만 섞어서 사용할 수 있도록 메서드만을 제공하는 유틸리티 형태의 클래스


In [49]:
class dbhandler_decorator(contextlib.ContextDecorator):
    def __enter__(self):
        stop_database()
        
    def __exit__(self, exec_type, ex_value, ex_traceback):
        start_database()
        
@dbhandler_decorator()
def offline_backup():
    run("pg_dump database")

**위의 코드는**  
- with 문을 사용하지 않음
- 함수를 호출하기만 하면 offline_backup 함수가 자동으로 컨텍스트 관리자 안에서 실행됨
- 컨텍스트 관리자 내부에서 사용하고자 하는 객체를 얻을 수 없음  
    ```python
    with offline_backup() as bp:
    ```
    위처럼 &#95;&#95;enter&#95;&#95; 메서드가 반환한 객체를 사용해야 하는 경우는 이전의 접근방식을 사용해야 함  
    
<hr>

### contextlib.suppress
- 제공한 예외 중 하나가 발생한 경우에는 실패하지 않도록 함
- try/except와 유사하나 suppress 메서드를 호출하면 로직에서 자체적으로 처리하고 있는 예외임을 명시함
- 아래의 예시에서 DataConversionException은 입력 데이터가 이미 기대한 것과 같은 포맷이어서 변환할 필요가 없으므로 무시해도 안전하다는 뜻

In [52]:
import contextlib

def main():
    with contextlib.suppress(DataConversionException):
        parse_data(input_json_or_dict)